In [2]:
with open(f'run_baseline_pNN.sh', 'w') as f:
    for ds in range(13):
        line = f'sbatch experiment.py --DATASET {ds} --skipconnection False --areaestimator none  --projectname Baseline\n'
        f.write(line)

In [3]:
import numpy as np
gammas = np.linspace(0,1,50)
gammas = np.round(gammas, 5)
gammas = np.unique(gammas)

for ds in range(13):
    with open(f'run_area_aware_training_{ds:02d}.sh', 'w') as f:
        for gamma in gammas:
            line = f'sbatch experiment.py --DATASET {ds} --skipconnection True --areaestimator area --areabalance {gamma:.5f}  --projectname AreaAwareTraining\n'
            f.write(line)

In [4]:
import numpy as np
gammas = np.linspace(0,1,50)
gammas = np.round(gammas, 5)
gammas = np.unique(gammas)

for ds in range(13):
    with open(f'run_area_aware_neat_{ds:02d}.sh', 'w') as f:
        for gamma in gammas:
            line = f'sbatch experiment_EA.py --DATASET {ds} --areabalance {gamma:.5f}  --projectname AwareAwareNEAT\n'
            f.write(line)

In [1]:
with open('evaluation_baseline_area.sh', 'w') as f:
    for ds in range(13):
        line = f'sbatch evaluation.py --DATASET {ds}  --areaestimator none --DEVICE cpu\n'
        f.write(line)

In [2]:
gammas = np.linspace(0,1,50)
gammas = np.round(gammas, 5)
gammas = np.unique(gammas)

In [3]:
gammas

array([0.     , 0.02041, 0.04082, 0.06122, 0.08163, 0.10204, 0.12245,
       0.14286, 0.16327, 0.18367, 0.20408, 0.22449, 0.2449 , 0.26531,
       0.28571, 0.30612, 0.32653, 0.34694, 0.36735, 0.38776, 0.40816,
       0.42857, 0.44898, 0.46939, 0.4898 , 0.5102 , 0.53061, 0.55102,
       0.57143, 0.59184, 0.61224, 0.63265, 0.65306, 0.67347, 0.69388,
       0.71429, 0.73469, 0.7551 , 0.77551, 0.79592, 0.81633, 0.83673,
       0.85714, 0.87755, 0.89796, 0.91837, 0.93878, 0.95918, 0.97959,
       1.     ])